# Visualization of the types of supports used by mobile carriers in France  

*Charles Laverdure - 26 July 2023*

## Introduction 


## Librairies 

In [4]:
%reset
import pandas as pd 
import numpy as np
import geopandas as gpd
import glob
import os

## Variables 

In [6]:
df_aggregated = pd.read_csv('data/anfr/20230720150852_observatoire_2g_3g_4g.csv', sep=';')
display(df_aggregated.head(2))

,id,adm_lb_nom,sup_id,emr_lb_systeme,emr_dt,sta_nm_dpt,code_insee,generation,date_maj,sta_nm_anfr,...,tpo_id,adr_lb_lieu,adr_lb_add1,adr_lb_add2,adr_lb_add3,adr_nm_cp,com_cd_insee,coordonnees,coord,statut
0,1,SFR,513700,LTE 1800,2016-12-20,031,31118,4G,2023-07-19,0312700486,...,65.0,LES HEBRAILS,RTE DE BOULOC,NaN,NaN,31620,31118,"43.793333333333, 1.375555555556",43°47'36''N 1°22'32''E,En service
1,2,SFR,513700,GSM 900,2015-12-09,031,31118,2G,2023-07-19,0312700486,...,65.0,LES HEBRAILS,RTE DE BOULOC,NaN,NaN,31620,31118,"43.793333333333, 1.375555555556",43°47'36''N 1°22'32''E,En service


## Code 

In [15]:
df_major_ops = df_aggregated[df_aggregated['adm_lb_nom'].isin(['ORANGE', 'SFR', 'FREE', 'BOUYGUES TELECOM'])]
df_major_ops = df_major_ops.drop(['sta_nm_dpt', 'date_maj', 'tpo_id', 'adr_lb_lieu', 'adr_lb_add1', 'adr_lb_add2', 'adr_lb_add3', 'adr_nm_cp', 'com_cd_insee', 'coord', 'statut'], axis = 1)
# transform coordonnees column into two columns (latitude and longitude)
df_major_ops[['latitude', 'longitude']] = df_major_ops['coordonnees'].str.split(',', expand=True)
df_major_ops = df_major_ops.drop(['coordonnees'], axis = 1)
display(df_major_ops.head(2))
display(df_major_ops.shape)

,id,adm_lb_nom,sup_id,emr_lb_systeme,emr_dt,code_insee,generation,sta_nm_anfr,nat_id,sup_nm_haut,latitude,longitude
0,1,SFR,513700,LTE 1800,2016-12-20,31118,4G,0312700486,23,46.5,43.793333333333,1.375555555556
1,2,SFR,513700,GSM 900,2015-12-09,31118,2G,0312700486,23,46.5,43.793333333333,1.375555555556


(515665, 12)

In [ ]:
# group by sta_nm_anfr and set an aggregation function
